# Vulnerability analysis of popular CNN models to untargeted evasion attacks (FGSM, PGD, BIM, JSMA)

**Install ART framwerok**

In [ ]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.6 MB/s eta 0:00:00


**Install dependecies**

In [ ]:
import numpy as np
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod, SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from art.utils import load_dataset
import torch.nn as nn
import torch.optim as optim

**Import data set and pretrained models**

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('cifar10')
x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)
y_train = np.argmax(y_train, axis=1)
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.201)

test_models = {

    'Resnet56': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet56", pretrained=True),
    'VGG19': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg19_bn", pretrained=True),
    'MobileNetV2': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_4", pretrained=True),
    'repvgg_a2': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_repvgg_a2", pretrained=True),
}


**Evaluate effectivness**

In [ ]:

# Wrapper function for evasion attacks
def attack(x_test, attack_class, classifier, **kwargs):
    attack_instance = attack_class(classifier, **kwargs)
    adversarial_examples = attack_instance.generate(x=x_test)
    return adversarial_examples



def evaluate_metrics(classifier, x_test, y_test):
  predictions = classifier.predict(x_test)
  predicted_classes = np.argmax(predictions, axis=1)
  accuracy = accuracy_score(y_test, predicted_classes)
  precision = precision_score(y_test, predicted_classes, average='macro')
  recall = recall_score(y_test, predicted_classes, average='macro')
  f1 = f1_score(y_test, predicted_classes, average='macro')

  print(f"  \nMiary dla: {model_name}:")
  print(f"  Dokładność: {accuracy:.4f}")
  print(f"  Precyzja: {precision:.4f}")
  print(f"  Czułość: {recall:.4f}")
  print(f"  F1: {f1:.4f}")
  print(f"\n")


for model_name,  model_fn in test_models.items():
    print(f"\nTrenowanie modelu: {model_name}")
    model = model_fn()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()
    classifier = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
        clip_values=(min_, max_),
        preprocessing = (mean, std)
    )

    evaluate_metrics(classifier, x_test, np.argmax(y_test, axis=1))
    print("FGSM: \n")
    fgsm_adver = attack(x_test, FastGradientMethod, classifier=classifier, eps=0.1, targeted=False)
    evaluate_metrics(classifier, fgsm_adver, np.argmax(y_test, axis=1))

    print("PGD: \n")
    pgd_adver = attack(x_test, ProjectedGradientDescent, classifier=classifier, eps=0.1, eps_step=0.01, max_iter=40, targeted=False)
    evaluate_metrics(classifier, pgd_adver, np.argmax(y_test, axis=1))

    print("BIM: \n")
    bim_adver = attack(x_test, BasicIterativeMethod, classifier=classifier, eps=0.01, eps_step=0.001, max_iter=10,
batch_size=32,  targeted=False)
    evaluate_metrics(classifier, bim_adver, np.argmax(y_test, axis=1))


    print("JSMA: \n")
    jacob_adver = attack(x_test, SaliencyMapMethod, classifier=classifier, theta=0.1, gamma=1, batch_size=32)
    evaluate_metrics(classifier, jacob_adver, np.argmax(y_test, axis=1))



Trenowanie modelu: Resnet56


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet56-187c023a.pt" to /root/.cache/torch/hub/checkpoints/cifar10_resnet56-187c023a.pt


100%|██████████| 3.39M/3.39M [00:00<00:00, 13.0MB/s]


  
Miary dla: Resnet56:
  Dokładność: 0.9437
  Precyzja: 0.9436
  Czułość: 0.9437
  F1: 0.9436


FGSM: 

  
Miary dla: Resnet56:
  Dokładność: 0.1512
  Precyzja: 0.3280
  Czułość: 0.1512
  F1: 0.1035


PGD: 



...

  
Miary dla: Resnet56:
  Dokładność: 0.0373
  Precyzja: 0.0389
  Czułość: 0.0373
  F1: 0.0354


BIM: 



PGD - Batches:   0%|          | 0/313 [00:00<?, ?it/s]

  
Miary dla: Resnet56:
  Dokładność: 0.1320
  Precyzja: 0.1486
  Czułość: 0.1320
  F1: 0.1373


JSMA: 



JSMA:   0%|          | 0/313 [00:00<?, ?it/s]

KeyboardInterrupt: 